In [1]:
from math import sqrt, log, pi, ceil, floor
import matplotlib.pyplot as plt

## Genric functions

def show(var_name, var):
    print(var_name, ' = ', round(var,2))

from earthing import *

In [2]:
### Input parameters

Un = 11000  # Source voltage
c = 1.1  # Voltage factor

In [3]:
### Fault current calculations

## Impedence calculations

# Source substation
S_Q = 350e6  # Short circuit impedence at substation (Fault level as per IEC-60076-5 for 11kV)
             # Nominal for 33kV as per Inspectorate handbook is < 300 MVA
RG_Q = 0.5

Z_Q_abs = c*Un**2/S_Q
X_Q = 0.995*Z_Q_abs
R_Q = 0.1*X_Q  # Considering an X/R of 10
Z_Q = R_Q + 1j*X_Q

# 110kV - campus check meter (Cable feeder 3x300 sqmm, 33kV)
L1 = 0.6 + 3.029  # Line length in km
Z1 = 0.13*L1 + 1j*0.1*L1

# Campus check meter - Main SS (Cable feeder 3x300 sqmm, 33kV)
L2 = 1.415  # Line length in km
Z2 = 0.13*L2 + 1j*0.1*L2

# Main SS - LCSS (Cable feeder 3x300 sqmm, 11kV)
L3 = 0.5  # Line length in km
Z3 = 0.13*L3 + 1j*0.1*L3


## Fault current calculations

# Fault current at 110kV SS
I_FQ = fault_current(Un, c, Z_Q)

# Fault current at check meter
I_F1 = fault_current(Un, c, Z_Q + Z1)

# Fault current at Main SS
I_F2 = fault_current(Un, c, Z_Q + Z1 + Z2)

# Fault current at LCSS
I_F3 = fault_current(Un, c, Z_Q + Z1 + Z2 + Z3)

show('I_FQ', I_FQ)
show('I_F1', I_F1)
show('I_F2', I_F2)
show('I_F3', I_F3)

I_FQ  =  18370.92
I_F1  =  7765.97
I_F2  =  6222.75
I_F3  =  5810.53


In [4]:
### Check meter

# Parameters
rho = 30  # Resistivity of premises in Ohm-m
rho_s = 10000  # Resistivity of surface layer in Ohm-m
h_s = 0.15  # Thickness of surface layer in m
t_f = 3  # Fault duration in seconds
t_s = 3  # Shock duration in seconds for calculation of step and touch potential limits
I_ratio_J = 0.4  # Current division factor for current loading calculation

Lx = 10
Ly = 5
Lr = 3
Nr = 6
Le = (Lx + Ly)*2
S = (Lx + Ly)*2 / Lr

Z = Z_Q + Z1
L = L1

# Current density
J = max_current_density(rho, t_f)
I_F = fault_current(Un, c, Z)
A = I_F * I_ratio_J / J
N_plate = ceil(A / (0.6*0.6*2))

# Grid resistance
# Rn = R_plate(rho, 0.6*0.6)
# R = Rn / N_plate
R = resistance_grid_with_rods(rho, A=Lx*Ly, L_E=Le, L_R=Lr, N=Nr, S=S, d=0.025, w=0.025)  # Use earth grid assumption to calculate resistance

# Fault current
I_ratio = current_ratio(rho, R + RG_Q, 130, Un/1000, 300, L)
I_gr = I_F * I_ratio

# Touch and step voltage calculations
EPR = I_gr * R
perm_touch_volt = e_touch_70(rho, rho_s, h_s, t_s)
perm_step_volt = e_step_70(rho, rho_s, h_s, t_s)
Em, Es = e_mesh_step_grid(rho, Lx, Ly, Lr, Nx=2, Ny=2, Nr=Nr, d=0.025/pi, h=0.5, Ig=I_gr)

show('J', J)
show('I_F', I_F)
show('I_ratio_J', I_ratio_J)
show('I_F (loading)', I_F*I_ratio_J)
show('A', A)
show('N_plate (required to meet J)', N_plate)
show('R', R)
print()
show('I_ratio', I_ratio)
show('I_gr', I_gr)
show('EPR', EPR)
show('Permissible touch voltage', perm_touch_volt)
show('Permissible step voltage', perm_step_volt)
show('Mesh voltage', Em)
show('Step voltage', Es)

J  =  797.95
I_F  =  7765.97
I_ratio_J  =  0.4
I_F (loading)  =  3106.39
A  =  3.89
N_plate (required to meet J)  =  6
R  =  1.7

I_ratio  =  0.27
I_gr  =  2082.02
EPR  =  3549.53
Permissible touch voltage  =  1137.48
Permissible step voltage  =  4277.98
Mesh voltage  =  999.69
Step voltage  =  564.15


In [5]:
### Main substation

# Parameters
rho = 30  # Resistivity of premises in Ohm-m
rho_s = 10000  # Resistivity of surface layer in Ohm-m
h_s = 0.15  # Thickness of surface layer in m
t_f = 3  # Fault duration in seconds
t_s = 3  # Shock duration in seconds for calculation of step and touch potential limits
I_ratio_J = 0.4  # Current division factor for current loading calculation

Lx = 70
Ly = 25
Lr = 3
Nr = 12
Le = (Lx + Ly)*2
S = (Lx + Ly)*2 / Lr

Z = Z_Q + Z1 + Z2
L = L1 + L2

# Current density
J = max_current_density(rho, t_f)
I_F = fault_current(Un, c, Z)
A = I_F * I_ratio_J / J
N_plate = ceil(A / (0.6*0.6*2))

# Grid resistance
# Rn = R_plate(rho, 0.6*0.6)
# R = Rn / N_plate
R = resistance_grid_with_rods(rho, A=Lx*Ly, L_E=Le, L_R=Lr, N=Nr, S=S, d=0.025, w=0.025)  # Use earth grid assumption to calculate resistance

# Fault current
I_ratio = current_ratio(rho, R + RG_Q, 130, Un/1000, 300, L)
I_gr = I_F * I_ratio

# Touch and step voltage calculations
EPR = I_gr * R
perm_touch_volt = e_touch_70(rho, rho_s, h_s, t_s)
perm_step_volt = e_step_70(rho, rho_s, h_s, t_s)
Em, Es = e_mesh_step_grid(rho, Lx, Ly, Lr, Nx=2, Ny=2, Nr=Nr, d=0.025/pi, h=0.5, Ig=I_gr)

show('J', J)
show('I_F', I_F)
show('I_ratio_J', I_ratio_J)
show('I_F (loading)', I_F*I_ratio_J)
show('A', A)
show('N_plate (required to meet J)', N_plate)
show('R', R)
print()
show('I_ratio', I_ratio)
show('I_gr', I_gr)
show('EPR', EPR)
show('Permissible touch voltage', perm_touch_volt)
show('Permissible step voltage', perm_step_volt)
show('Mesh voltage', Em)
show('Step voltage', Es)

J  =  797.95
I_F  =  6222.75
I_ratio_J  =  0.4
I_F (loading)  =  2489.1
A  =  3.12
N_plate (required to meet J)  =  5
R  =  0.4

I_ratio  =  0.35
I_gr  =  2185.8
EPR  =  878.23
Permissible touch voltage  =  1137.48
Permissible step voltage  =  4277.98
Mesh voltage  =  416.36
Step voltage  =  118.31


In [6]:
### LCSS

# Parameters
rho = 30  # Resistivity of premises in Ohm-m
rho_s = 10000  # Resistivity of surface layer in Ohm-m
h_s = 0.15  # Thickness of surface layer in m
t_f = 3  # Fault duration in seconds
t_s = 3  # Shock duration in seconds for calculation of step and touch potential limits
I_ratio_J = 0.4  # Current division factor for current loading calculation

Lx = 18
Ly = 3
Lr = 3
Nr = 6
Le = (Lx + Ly)*2
S = (Lx + Ly)*2 / Lr

Z = Z_Q + Z1 + Z2 + Z3
L = L1 + L2 + L3

# Current density
J = max_current_density(rho, t_f)
I_F = fault_current(Un, c, Z)
A = I_F * I_ratio_J / J
N_plate = ceil(A / (0.6*0.6*2))

# Grid resistance
# Rn = R_plate(rho, 0.6*0.6)
# R = Rn / N_plate
R = resistance_grid_with_rods(rho, A=Lx*Ly, L_E=Le, L_R=Lr, N=Nr, S=S, d=0.025, w=0.025)  # Use earth grid assumption to calculate resistance

# Fault current
I_ratio = current_ratio(rho, R + RG_Q, 130, Un/1000, 300, L)
I_gr = I_F * I_ratio

# Touch and step voltage calculations
EPR = I_gr * R
perm_touch_volt = e_touch_70(rho, rho_s, h_s, t_s)
perm_step_volt = e_step_70(rho, rho_s, h_s, t_s)
Em, Es = e_mesh_step_grid(rho, Lx, Ly, Lr, Nx=2, Ny=2, Nr=Nr, d=0.025/pi, h=0.5, Ig=I_gr)

show('J', J)
show('I_F', I_F)
show('I_ratio_J', I_ratio_J)
show('I_F (loading)', I_F*I_ratio_J)
show('A', A)
show('N_plate (required to meet J)', N_plate)
show('R', R)
print()
show('I_ratio', I_ratio)
show('I_gr', I_gr)
show('EPR', EPR)
show('Permissible touch voltage', perm_touch_volt)
show('Permissible step voltage', perm_step_volt)
show('Mesh voltage', Em)
show('Step voltage', Es)

J  =  797.95
I_F  =  5810.53
I_ratio_J  =  0.4
I_F (loading)  =  2324.21
A  =  2.91
N_plate (required to meet J)  =  5
R  =  1.67

I_ratio  =  0.31
I_gr  =  1788.39
EPR  =  2989.61
Permissible touch voltage  =  1137.48
Permissible step voltage  =  4277.98
Mesh voltage  =  838.48
Step voltage  =  405.45
